In [15]:
# NN 4 outputs and with and without sorting of outputs
model_path = 'saved_model/Multi_leak/experiment4/Mask/'
project_name='Multi_leak_experiment4_Mask_withSwap'

In [16]:
# Experiment 2 - 1 output layer with 1 loss function - mse. and do hyper parameter tuning.
from utils.data_preprocess import load_data
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt
from keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from kerastuner import HyperModel, Hyperband
from keras import backend as backend

In [17]:
single_leakage, two_leakage = load_data()
# two_leakage["leak_1"] = 1
# two_leakage["leak_2"] = 1

# single_leakage["leak_1"] = 1
# single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
data['x2'] = data['x2'].replace(np.nan, 0)
data['y2'] = data['y2'].replace(np.nan, 0)

data = data.drop(columns=['mfc6_residual',
       'mfc7_residual', 'mfc8_residual', 'mfc9_residual', 'mfc10_residual',
       'mfc1_residual', 'mfc2_residual', 'mfc3_residual', 'mfc4_residual',
       'mfc5_residual', 'total flow rate'
       ])

y = data[['x1', 'y1', 'x2', 'y2']]
x = data.drop(['x1', 'y1', 'x2', 'y2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 


# sort x1, y1 and x2, y2. coordinates with lowest x will take the position of x1
def coords_swap(y1):
    s = y1['x2'] < y1['x1']
    y1.loc[s, ['x1','x2']] = y1.loc[s, ['x2','x1']].values
    y1.loc[s, ['y1','y2']] = y1.loc[s, ['y2','y1']].values
    return y1


y1_data = [y_train, y_val, y_test]
y1_data_types = ['y_train', 'y_val', 'y_test']
for y1_data_types, y1 in zip(y1_data_types, y1_data):
    y1_data_types = coords_swap(y1)

y1_train = y_train[['x1', 'y1']]
y1_test = y_test[['x1', 'y1']]
y1_val = y_val[['x1', 'y1']]

y2_train = y_train[['x2', 'y2']]
y2_test = y_test[['x2', 'y2']]
y2_val = y_val[['x2', 'y2']]

y1_columns = y1_train.columns
y2_columns = y2_train.columns
X_columns = X_train.columns

scaler_coords1 = StandardScaler()
y1_train = scaler_coords1.fit_transform(y1_train)
y1_test = scaler_coords1.transform(y1_test)
y1_val = scaler_coords1.transform(y1_val)

y1_train = pd.DataFrame(y1_train, columns=y1_columns)
y1_test = pd.DataFrame(y1_test, columns=y1_columns)
y1_val = pd.DataFrame(y1_val, columns=y1_columns)

scaler_coords2 = StandardScaler()
y2_train = scaler_coords2.fit_transform(y2_train)
y2_test = scaler_coords2.transform(y2_test)
y2_val = scaler_coords2.transform(y2_val)

y2_train = pd.DataFrame(y2_train, columns=y2_columns)
y2_test = pd.DataFrame(y2_test, columns=y2_columns)
y2_val = pd.DataFrame(y2_val, columns=y2_columns)

scaler_flows = StandardScaler()
X_train = scaler_flows.fit_transform(X_train)
X_test = scaler_flows.transform(X_test)
X_val = scaler_flows.transform(X_val)

X_train = pd.DataFrame(X_train, columns=X_columns)
X_test = pd.DataFrame(X_test, columns=X_columns)
X_val = pd.DataFrame(X_val, columns=X_columns)

# y_train = [y1_train, y2_train]
# y_val = [y1_val, y2_val]
# y_test = [y1_test, y2_test]

y_train_all = pd.concat([y1_train, y2_train], axis=1)
y_test_all = pd.concat([y1_test, y2_test], axis=1)
y_val_all = pd.concat([y1_val, y2_val], axis=1)

In [18]:
X_train_np, y1_train_np, y2_train_np = X_train.values, y1_train.values, y2_train.values
X_val_np, y1_val_np, y2_val_np = X_val.values, y1_val.values, y2_val.values
X_test_np, y1_test_np, y2_test_np = X_test.values, y1_test.values, y2_test.values

# Create TensorFlow datasets from NumPy arrays.
batch_size = 32
buffer_size = len(X_train)  # Set the buffer size to the number of training examples for full shuffling.

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y1_train_np, y2_train_np ))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_np, y1_val_np, y2_val_np))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_np, y1_test_np, y2_test_np))

# Shuffle, batch, and prefetch the training dataset.
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Batch the validation and test datasets.
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [19]:
def masked_mse(y_true, y_pred):
    mask = keras.backend.cast(keras.backend.not_equal(y_true, 0), keras.backend.floatx())
    mse = keras.backend.mean(keras.backend.square(y_true - y_pred) * mask)
    return mse

losses = {
	"y1": "mse",
	"y2": masked_mse
    # "y2" : 'mse'
    }

metrics = {
    "y1": 'mae',
    "y2": 'mae'
    }

In [20]:
EPOCHS = 1000

# Define custom loss function


# Define model builder function for Keras Tuner
def model_builder(hp):
    tf.keras.backend.clear_session()
    # Tune the number of units in the first Dense layer
    # Define model
    inputs = keras.Input(shape=(10,))
    shared_layer = inputs
    for i in range(hp.Int('num_layers', 1, 15)):
        shared_layer = layers.Dense(
            units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
            activation='relu',
            # add elu
            kernel_initializer='he_uniform'
        )(shared_layer)

    out1 = shared_layer
    out1 = layers.Dense(2, name='y1', activation = 'linear', kernel_initializer='he_uniform')(out1)

    out2 = shared_layer
    out2 = layers.Dense(2, name='y2', activation = 'linear', kernel_initializer='he_uniform')(out2)

    model = Model(inputs=inputs, outputs=[out1, out2])

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-1, sampling="log")

    # model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss=['mse', masked_mse])
    model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=hp_learning_rate),
                    loss=losses, metrics=metrics)
    
    return model

tuner = Hyperband(
    model_builder,
    objective =  kt.Objective("val_loss", direction="min"),
    max_epochs=EPOCHS+100,
    factor=2,
    directory="../../tensorflow_log_files/studienarbeit/",
    project_name=project_name,
    seed = 0
)

tuner.search_space_summary()
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Perform hyperparameter search
tuner.search(train_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})),
            #  X_train, y_train, 
            #  validation_data = (X_val, y_val), 
            validation_data = val_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})), 
             verbose = 1, callbacks=[stop_early],
             epochs=EPOCHS, shuffle = True)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:", best_hps)

# Build the best model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the best model on the full dataset
history = best_model.fit(train_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})),
            #  X_train, y_train,
            #  validation_data = (X_val, y_val), 
            validation_data = val_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})),  
            callbacks=[stop_early],
            verbose = 1, epochs=EPOCHS, shuffle = True)

print(f"""
    The hyperparameter search is complete. The optimal learning rate for the optimizer
    is {best_model.optimizer.lr.numpy()}.
    """)

Trial 11 Complete [00h 00m 07s]
val_loss: 0.27254274909196285

Best val_loss So Far: 0.27254274909196285
Total elapsed time: 00h 01m 27s

Search: Running Trial #12

Value             |Best Value So Far |Hyperparameter
13                |8                 |num_layers
64                |512               |units_0
9                 |7                 |task_0_num_layers
0.017146          |0.0032825         |lr
96                |160               |units_1
288               |256               |units_2
1                 |8                 |task_2_num_layers
64                |64                |units_3
288               |96                |units_4
480               |128               |units_5
480               |32                |units_6
512               |192               |units_7
224               |352               |units_8
1                 |9                 |task_8_num_layers
64                |24                |task_2_layer_0_neurons
64                |32                |task_2_laye

In [ ]:
best_model.save(model_path)
best_model = tf.keras.models.load_model(model_path)
best_model.summary()

In [ ]:
y_predictions_train = best_model.predict(train_dataset.map(lambda x, y1: (x, {'y1': y1})))
# print("train", "{:10.4f}".format(mean_squared_error(y_train, y_predictions, squared=True)))
y_predictions_val = best_model.predict(val_dataset.map(lambda x, y1: (x, {'y1': y1})))
# print("val", "{:10.4f}".format(mean_squared_error(y_val, y_predictions, squared=True)))
y_predictions = best_model.predict(test_dataset.map(lambda x, y1: (x, {'y1': y1})))


loss_test = "{:10.4f}".format(mean_squared_error(y_test_all, y_predictions, squared=True))
metric_test = "{:10.4f}".format(mean_absolute_error(y_test_all, y_predictions))

loss_val = "{:10.4f}".format(mean_squared_error(y_val_all, y_predictions_val, squared=True))
metric_val = "{:10.4f}".format(mean_absolute_error(y_val_all, y_predictions_val))

loss_train = "{:10.4f}".format(mean_squared_error(y_train_all, y_predictions_train, squared=True))
metric_train = "{:10.4f}".format(mean_absolute_error(y_train_all, y_predictions_train))

print(metric_test, metric_val, metric_train)

In [ ]:
results_train = best_model.evaluate(train_dataset.map(lambda x, y1: (x, {'y1': y1})), verbose=1)
results_val = best_model.evaluate(val_dataset.map(lambda x, y1: (x, {'y1': y1})), verbose=1)
results_test = best_model.evaluate(test_dataset.map(lambda x, y1: (x, {'y1': y1,})), verbose=1)